In [1]:
import sys
sys.path.append('..')
import os
from os.path import expanduser

import lcatools

In [2]:
from lcatools.providers.ecoinvent_spreadsheet import EcoinventSpreadsheet

In [3]:
ECOINVENT_VERSION = '3.2'

SYSTEM_MODELS = ['undefined', 'apos', 'consequential', 'cut-off']

datapath = os.path.join(expanduser('~'), 'Dropbox', 'data', 'Ecoinvent', ECOINVENT_VERSION)
catalog_dir = os.path.join(os.path.expanduser('~'), 'GitHub', 'lca-tools-datafiles', 'catalogs')

In [6]:
ei = dict()
for model in SYSTEM_MODELS:
    pub_file = os.path.join(datapath, 'activity_overview_for_users_' + ECOINVENT_VERSION + '_' + model + '.xlsx')
    ei[model] = EcoinventSpreadsheet(pub_file, internal=False, version=ECOINVENT_VERSION)
    

In [7]:
ei

{'apos': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7fd552d3a0b8>,
 'consequential': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7fd552d3a160>,
 'cut-off': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7fd552d3a1d0>,
 'undefined': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7fd552d3a0f0>}

In [12]:
for model in SYSTEM_MODELS[2:5]:
    ei[model]._quiet=True
    ei[model].load_all()

Loading /home/b/Dropbox/data/Ecoinvent/3.2/activity_overview_for_users_3.2_consequential.xlsx
Reading elementary exchanges ...
Reading intermediate exchanges ...
20 new quantity entities added (20 total)
Handling elementary exchanges [public spreadsheet]
4013 new flow entities added (4013 total)
Handling intermediate exchanges [public spreadsheet]
2754 new flow entities added (6767 total)
Handling activities...
Reading activity overview ...
11468 new process entities added (11468 total)
Loading /home/b/Dropbox/data/Ecoinvent/3.2/activity_overview_for_users_3.2_cut-off.xlsx
Reading elementary exchanges ...
Reading intermediate exchanges ...
20 new quantity entities added (20 total)
Handling elementary exchanges [public spreadsheet]
4013 new flow entities added (4013 total)
Handling intermediate exchanges [public spreadsheet]
2754 new flow entities added (6767 total)
Handling activities...
Reading activity overview ...
11578 new process entities added (11578 total)


In [13]:
for k, v in ei.items():
    fname = 'ecoinvent_%s_%s_xlsx.json.gz' % (ECOINVENT_VERSION, k)
    v.write_to_file(os.path.join(catalog_dir, fname), exchanges=True, gzip=True)

In [7]:
ei

{'apos': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7f420d073a20>,
 'consequential': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7f420d073a90>,
 'cut-off': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7f420d073b00>,
 'undefined': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7f420d0739e8>}